# TDM Proquest Article Compression & Download

### Journal List
- Earth System Science Data
- Earth's Future
- Environmental Science and Pollution Research
- Frontiers in Marine Science
- Global Change Biology
- Water Resources Research
- Biotechnology for Biofuels
- Elementa
- Ecological Economics
- Food and Energy Security
- International Journal of Energy Research
- The Journal of Material Cycles and Waste Management
- Mitigation and Adaptation Strategies for Global Change
- Journal of Industrial Economics
- Climate Policy
- Clean Technologies and Environmental Policy
- Wind Energy Science
- Forests

### Specifications
**Date Range:** Jan 1st 2023 - Feb 15th 2024\
**Document Type:** Article

Code Authors: Kathryn Link-Oberstar, Chanteria Milner, Kate Habich

In [ ]:
# Installs and imports

%conda install xmltodict

import xmltodict
import os
import json
import pandas as pd

In [ ]:
# TODO: Replace 'dataset_name' with name of downloaded dataset
dataset_name = 'Reduced-Env-Dataset'
files_to_export = 'data/' + dataset_name
parq_file_path = 'data/ES-journals.parquet'

In [ ]:
# convert folder of xml to list of JSON files

file_list = os.listdir(files_to_export)
file_list = [file for file in file_list if file.endswith("xml")]

# TODO: may need to split up data here if > 15MB

list_of_file_data = []
for file in file_list:
    file_path = os.path.join(files_to_export, file)
    with open(file_path, "r") as f:
        file_as_xml = f.read()
        file_as_dict = xmltodict.parse(file_as_xml)
        list_of_file_data.append(json.dumps(file_as_dict))


In [ ]:
# Write to parquet and compress

df = pd.DataFrame(list_of_file_data)
df.columns = ["Data"]
df.to_parquet(parq_file_path, compression = 'BROTLI')

In [ ]:
# Calculate file size (must be <15MB)

file_size = os.path.getsize(parq_file_path)/1000000
print(f"File Size: {file_size} MB")
if file_size <= 15:
    print("File passes size requirements.")
else:
    print("File FAILS size requirements.")

In [ ]:
# Export and download selected journals

!aws s3 cp $parq_file_path s3://pq-tdm-studio-results/tdm-ale-data/a272/results/